# Comparison of aggregation

This notebook compares the 3 main methods of aggregation (Rank, Mean and Max).

Results are explained in Section 4.4.

In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
from utils import load_nested_results

all_results = load_nested_results("results/")

In [3]:
from itertools import combinations
from sklearn.metrics import average_precision_score
import numpy as np
from pyod.utils.utility import standardizer
from scipy.stats import rankdata

scores_all_types = {}

for dataset_name, results in all_results.items():

    score_dataset = {}
    model_names = list(results.keys() - {"ground_truth"})
    n_fold = len(results["ground_truth"])

    for model in model_names:
        scores_model = []
        for fold in range(n_fold):
            scores_model.append(
                average_precision_score(
                    results["ground_truth"][fold], results[model][fold]["scores"]
                )
            )
        score_dataset[model] = scores_model

    for comb in combinations(range(len(model_names)), 3):

        name1 = model_names[comb[0]]
        name2 = model_names[comb[1]]
        name3 = model_names[comb[2]]

        scores_model_rank = []
        scores_model_max = []
        scores_model_mean = []

        for fold in range(n_fold):
            y_true = results["ground_truth"][fold]

            scores_1 = results[name1][fold]["scores"]
            scores_2 = results[name2][fold]["scores"]
            scores_3 = results[name3][fold]["scores"]

            ranks = [
                rankdata(score, "average") for score in [scores_1, scores_2, scores_3]
            ]
            auc_rank = average_precision_score(y_true, np.mean(ranks, axis=0))
            scores_model_rank.append(auc_rank)

            scores_triplet_std = standardizer(np.stack([scores_1, scores_2, scores_3]))
            scores_agreges_max = np.max(scores_triplet_std, axis=0)
            auc_max = average_precision_score(y_true, scores_agreges_max)
            scores_model_max.append(auc_max)

            scores_triplet_std = standardizer(np.stack([scores_1, scores_2, scores_3]))
            scores_agreges_max = np.mean(scores_triplet_std, axis=0)
            auc_max = average_precision_score(y_true, scores_agreges_max)
            scores_model_mean.append(auc_max)

        score_dataset[f"{name1}-{name2}-{name3}_rank"] = scores_model_rank
        score_dataset[f"{name1}-{name2}-{name3}_max"] = scores_model_max
        score_dataset[f"{name1}-{name2}-{name3}_mean"] = scores_model_mean

    scores_all_types[dataset_name] = score_dataset

In [4]:
n_emsemble_rank = 0
n_emsemble_max = 0
n_emsemble_mean = 0

for model_name in scores_all_types[dataset_name].keys():
    if "_rank" in model_name:
        n_emsemble_rank += 1
    elif "_max" in model_name:
        n_emsemble_max += 1
    elif "_mean" in model_name:
        n_emsemble_mean += 1

In [6]:
import pandas as pd

all_res = {}


for dataset_name, dataset_scores in scores_all_types.items():
    means = np.zeros(4)

    for model_name, scores in dataset_scores.items():
        mean_score = np.max(scores)
        if "_rank" in model_name:
            means[1] += mean_score
        elif "_max" in model_name:
            means[2] += mean_score
        elif "_mean" in model_name:
            means[3] += mean_score
        else:
            means[0] += mean_score

    means[1] /= 364
    means[2] /= 364
    means[3] /= 364
    means[0] /= len(model_names)

    all_res[dataset_name] = means


df = (
    pd.DataFrame(
        all_res,
        index=["Unique", "Rank", "Max", "Mean"],
    )
    .round(2)
    .T
)

df[["Rank", "Max", "Mean"]] *= 100
df[["Rank", "Max", "Mean"]].round()

,Rank,Max,Mean
2_annthyroid,26.0,21.0,9.0
4_breastw,98.0,59.0,48.0
14_glass,18.0,22.0,19.0
15_Hepatitis,84.0,64.0,48.0
21_Lymphography,100.0,68.0,36.0
23_mammography,30.0,8.0,4.0
27_PageBlocks,60.0,35.0,12.0
29_Pima,55.0,45.0,42.0
37_Stamps,62.0,35.0,22.0
38_thyroid,58.0,28.0,5.0


In [7]:
# print(df.mean().to_latex(float_format="%.0f"))
df[["Rank", "Max", "Mean"]].mean().round()

Rank    52.0
Max     35.0
Mean    24.0
dtype: float64